In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import _tree
from sklearn import tree as sktree
import graphviz

In [2]:
# load iris dataset, train RFC
iris = load_iris() 
X = iris['data']
y = iris['target']

model= RandomForestClassifier(random_state=0)
model.fit(X, y)

RandomForestClassifier(random_state=0)

In [3]:
n = 52  # határeset
x = X[n:n+1]
tree_idx = 0
n_classes = 3
n_trees = 100

In [4]:
# adott mintára fa szavazatok
def tree_votes(x):
    votes = np.zeros(n_trees, dtype=int)
    for idx, tree in enumerate(model.estimators_):
        votes[idx] = tree.predict(x)
    return votes

def count_votes(x):
    votes = np.zeros(n_classes)
    for tree in model.estimators_:
        votes[int(tree.predict(x))] += 1
    return votes

In [5]:
votes = tree_votes(x)
expected = model.predict(x)[0]

# egyes fákon milyen featureök mentén haladunk végig
def recurse(idx, tree, node, f_list):
    if tree.feature[node] != _tree.TREE_UNDEFINED:
        # if threshold is close to our value, add it to list
        feature = tree.feature[node]
        threshold = tree.threshold[node]
        input_val = x[0,feature]
        diff = threshold-input_val
        # only add close values
        if (abs(diff) <= 0.1):
            f_list.append((feature, input_val, threshold, diff))
        if (input_val <= threshold):
            recurse(idx, tree, tree.children_left[node], f_list)
        else:
            recurse(idx, tree, tree.children_right[node], f_list)
    else:
        output = np.argmax(tree.value[node]) 
        info.append((idx, output, f_list.copy()))

info = []
feature_names = ['SL', 'SW', 'PL', 'PW']
for idx, tree in enumerate(model.estimators_):
    feature_list = []
    t = tree.tree_
    recurse(idx, t, 0, feature_list)


In [6]:
for tuple in info:
    idx, output, line = tuple
    for data in line:
        fidx, val, th, diff = data
        feature = feature_names[fidx]
        print("Tree: {} out: {}  feature: {}  {} <= {:4.5}, diff: {:4.5}".format(idx, output, feature, val, th, diff))

Tree: 0 out: 1  feature: PL  4.9 <= 4.85, diff: -0.05
Tree: 1 out: 1  feature: PL  4.9 <= 4.95, diff: 0.05
Tree: 2 out: 1  feature: PW  1.5 <= 1.55, diff: 0.05
Tree: 2 out: 1  feature: PL  4.9 <=  5.0, diff:  0.1
Tree: 3 out: 1  feature: PL  4.9 <= 4.95, diff: 0.05
Tree: 4 out: 1  feature: PL  4.9 <= 4.95, diff: 0.05
Tree: 5 out: 1  feature: PL  4.9 <= 4.95, diff: 0.05
Tree: 5 out: 1  feature: PW  1.5 <=  1.6, diff:  0.1
Tree: 9 out: 1  feature: PL  4.9 <= 4.95, diff: 0.05
Tree: 10 out: 1  feature: PL  4.9 <= 4.95, diff: 0.05
Tree: 11 out: 1  feature: PL  4.9 <= 4.95, diff: 0.05
Tree: 11 out: 1  feature: SW  3.1 <= 3.05, diff: -0.05
Tree: 12 out: 1  feature: PL  4.9 <=  5.0, diff:  0.1
Tree: 13 out: 1  feature: PL  4.9 <=  4.9, diff: 9.5367e-08
Tree: 14 out: 1  feature: SL  6.9 <=  7.0, diff:  0.1
Tree: 14 out: 1  feature: PL  4.9 <=  5.0, diff:  0.1
Tree: 15 out: 1  feature: PL  4.9 <= 4.85, diff: -0.05
Tree: 16 out: 1  feature: PL  4.9 <= 4.95, diff: 0.05
Tree: 16 out: 1  feature: PL

In [7]:
# modding

print("Eredeti értékek:")

print(x[0])
x_ = x+[0,0,0.15,0] # 3. featurehöz +0.15 (3%os módosítás)

print("\nMódosított értékek:")
print(x_[0])

ratio = (1-(np.linalg.norm(x) / np.linalg.norm(x_)))*100
print("\nMódosítás mértéke: {:.2}%".format(ratio))
# diff

print("\nFa szavazatok:")
print(count_votes(x))
print(count_votes(x_))

Eredeti értékek:
[6.9 3.1 4.9 1.5]

Módosított értékek:
[6.9  3.1  5.05 1.5 ]

Módosítás mértéke: 0.88%

Fa szavazatok:
[ 0. 96.  4.]
[ 0. 40. 60.]
